In [1]:
import torch
import torchvision.models as model
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, datasets
import os
import glob
from tqdm import tqdm
from PIL import Image
import random
from sklearn.model_selection import train_test_split
import numpy as np
import timm.models as tmodels

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
Res=model.resnet18(pretrained=False)
X=tmodels.xception(pretrained=False)

for Model in [Res,X]:
    num_ftrs = Model.fc.in_features
    Model.fc = nn.Linear(num_ftrs,2)

In [4]:
cos=nn.CosineSimilarity(dim=1).cuda()
def ret_cos(x,y):
    return cos(x,y).mean().item()

In [5]:
from time import time

In [8]:
len(train_loader) * 32

48320

In [6]:
from sklearn.metrics import classification_report

cnt=-1
cnt_m=-1
model_name=['R18','X']
train_name=['D','P']
child=[-1,-4]
w,h=178,218
b_size = 32
transform = transforms.Compose([
    transforms.Resize((w,h)),
    transforms.ToTensor()
])

    
for train_path in ['/media/data1/dataset/DFDC/train','/media/data1/dataset/PGGAN/256x256/train']:
    cnt+=1
    for test in range(1,6):
        test_path = f'/media/data2/rjsdn/zoom/dataset/test{test}/'
        
        train_img = datasets.ImageFolder(train_path,transform=transform)
        test_img = datasets.ImageFolder(test_path,transform=transform)

        train_loader = DataLoader(train_img,batch_size=b_size,shuffle=False,num_workers=4,drop_last=True)
        test_loader = DataLoader(test_img,batch_size=b_size,shuffle=False,num_workers=4)
        
        for model in [Res,X]:
            s = time()
            cnt_m+=1
            if cnt_m>=1: continue
            model.load_state_dict(torch.load(f'weights/{model_name[cnt_m%2]}_{train_name[cnt%2]}/best.pth'))
            model = nn.Sequential(*(list(model.children())[:child[cnt_m%2]]))
            model.cuda()
            model.eval()
            f = open(f'{model_name[cnt_m%2]}_{train_name[cnt%2]}_test{test}_cosine.txt','w')
            with torch.no_grad():
                for x,y in train_loader:
                    x=x.cuda()
                    output = model(x).view(b_size,-1)
                    for tx,ty in tqdm(test_loader):
                        tx=tx.cuda()
                        tout = model(tx).view(b_size,-1)
                        res = ret_cos(output,tout)
                        f.write(f'{res}\n')
            f.close()
            print('time',time()-s)




100%|██████████| 125/125 [00:04<00:00, 25.80it/s]


100%|██████████| 125/125 [00:05<00:00, 24.42it/s]


100%|██████████| 125/125 [00:05<00:00, 24.58it/s]


100%|██████████| 125/125 [00:05<00:00, 24.29it/s]


100%|██████████| 125/125 [00:05<00:00, 24.16it/s]


100%|██████████| 125/125 [00:05<00:00, 23.98it/s]


time 8832.748224973679


In [40]:
len(total)

400000

In [41]:
len(total)

400000